# SingleStoreVectorStore

>[SingleStore](https://singlestore.com/) 是一个强大、高性能的分布式 SQL 数据库解决方案，专为在 [云](https://www.singlestore.com/cloud/) 和本地环境中都能表现出色而设计。它拥有多功能的功能集，提供无缝的部署选项，同时实现无与伦比的性能。

SingleStore 的一个突出特点是其对向量存储和操作的先进支持，这使其成为需要复杂 AI 功能（例如文本相似性匹配）的应用程序的理想选择。通过内置的向量函数，如 [dot_product](https://docs.singlestore.com/managed-service/en/reference/sql-reference/vector-functions/dot_product.html) 和 [euclidean_distance](https://docs.singlestore.com/managed-service/en/reference/sql-reference/vector-functions/euclidean_distance.html)，SingleStore 使开发人员能够高效地实现复杂的算法。

对于热衷于在 SingleStore 中利用向量数据的开发人员来说，这里提供了一个全面的教程，指导他们了解 [处理向量数据](https://docs.singlestore.com/managed-service/en/developer-resources/functional-extensions/working-with-vector-data.html) 的复杂性。本教程深入介绍了 SingleStoreDB 中的 Vector Store，展示了其根据向量相似性进行搜索的能力。利用向量索引，可以以极快的速度执行查询，从而快速检索相关数据。

此外，SingleStore 的 Vector Store 无缝集成了 [基于 Lucene 的全文索引](https://docs.singlestore.com/cloud/developer-resources/functional-extensions/working-with-full-text-search/)，从而实现了强大的文本相似性搜索。用户可以根据文档元数据对象选定的字段过滤搜索结果，从而提高查询精度。

SingleStore 的独特之处在于它能够以各种方式结合向量搜索和全文搜索，提供灵活性和多功能性。无论是按文本或向量相似性预先过滤并选择最相关的数据，还是采用加权求和方法计算最终相似度得分，开发人员都有多种选择。

总之，SingleStore 提供了一个用于管理和查询向量数据的全面解决方案，为 AI 驱动的应用程序提供了无与伦比的性能和灵活性。

| Class | Package | JS support |
| :--- | :--- |  :---: |
| SingleStoreVectorStore | langchain_singlestore | ✅ | 

:::note

对于 langchain-community 版本 `SingleStoreDB`（已弃用），请参阅
[v0.2 文档](https://python.langchain.com/v0.2/docs/integrations/vectorstores/singlestoredb/)。

:::

## 设置

要访问 SingleStore 向量存储，您需要安装 `langchain-singlestore` 集成包。

## 初始化

要初始化 `SingleStoreVectorStore`，你需要一个 `Embeddings` 对象和 SingleStore 数据库的连接参数。

### 所需参数：
- **embedding** (`Embeddings`): 文本嵌入模型。

### 可选参数：
- **distance_strategy** (`DistanceStrategy`): 用于计算向量距离的策略。默认为 `DOT_PRODUCT`。选项：
  - `DOT_PRODUCT`: 计算两个向量的标量积。
  - `EUCLIDEAN_DISTANCE`: 计算两个向量之间的欧几里得距离。

- **table_name** (`str`): 表名。默认为 `embeddings`。
- **content_field** (`str`): 用于存储内容的字段。默认为 `content`。
- **metadata_field** (`str`): 用于存储元数据的字段。默认为 `metadata`。
- **vector_field** (`str`): 用于存储向量的字段。默认为 `vector`。
- **id_field** (`str`): 用于存储 ID 的字段。默认为 `id`。

- **use_vector_index** (`bool`): 启用向量索引（需要 SingleStore 8.5+）。默认为 `False`。
- **vector_index_name** (`str`): 向量索引的名称。如果 `use_vector_index` 为 `False`，则忽略此参数。
- **vector_index_options** (`dict`): 向量索引的选项。如果 `use_vector_index` 为 `False`，则忽略此参数。
- **vector_size** (`int`): 向量的大小。如果 `use_vector_index` 为 `True`，则此参数是必需的。

- **use_full_text_search** (`bool`): 在内容上启用全文索引。默认为 `False`。

### 连接池参数：
- **pool_size** (`int`): 连接池中的活动连接数。默认为 `5`。
- **max_overflow** (`int`): `pool_size` 之外的最大连接数。默认为 `10`。
- **timeout** (`float`): 连接超时（秒）。默认为 `30`。

### 数据库连接参数：
- **host** (`str`): 数据库的主机名、IP 或 URL。
- **user** (`str`): 数据库用户名。
- **password** (`str`): 数据库密码。
- **port** (`int`): 数据库端口。默认为 `3306`。
- **database** (`str`): 数据库名。

### 其他选项：
- **pure_python** (`bool`): 启用纯 Python 模式。
- **local_infile** (`bool`): 允许本地文件上传。
- **charset** (`str`): 字符串值的字符集。
- **ssl_key**, **ssl_cert**, **ssl_ca** (`str`): SSL 文件的路径。
- **ssl_disabled** (`bool`): 禁用 SSL。
- **ssl_verify_cert** (`bool`): 验证服务器的证书。
- **ssl_verify_identity** (`bool`): 验证服务器的身份。
- **autocommit** (`bool`): 启用自动提交。
- **results_type** (`str`): 查询结果的结构（例如 `tuples`、`dicts`）。

In [ ]:
import os

from langchain_singlestore.vectorstores import SingleStoreVectorStore

os.environ["SINGLESTOREDB_URL"] = "root:pass@localhost:3306/db"

vector_store = SingleStoreVectorStore(embeddings=embeddings)

## 管理向量存储

`SingleStoreVectorStore` 假定文档的 ID 是一个整数。以下是如何管理向量存储的示例。

### 向向量存储添加项目

您可以如下向向量存储添加文档：

In [ ]:
%pip install -qU langchain-core

In [ ]:
from langchain_core.documents import Document

docs = [
    Document(
        page_content="""In the parched desert, a sudden rainstorm brought relief,
            as the droplets danced upon the thirsty earth, rejuvenating the landscape
            with the sweet scent of petrichor.""",
        metadata={"category": "rain"},
    ),
    Document(
        page_content="""Amidst the bustling cityscape, the rain fell relentlessly,
            creating a symphony of pitter-patter on the pavement, while umbrellas
            bloomed like colorful flowers in a sea of gray.""",
        metadata={"category": "rain"},
    ),
    Document(
        page_content="""High in the mountains, the rain transformed into a delicate
            mist, enveloping the peaks in a mystical veil, where each droplet seemed to
            whisper secrets to the ancient rocks below.""",
        metadata={"category": "rain"},
    ),
    Document(
        page_content="""Blanketing the countryside in a soft, pristine layer, the
            snowfall painted a serene tableau, muffling the world in a tranquil hush
            as delicate flakes settled upon the branches of trees like nature's own 
            lacework.""",
        metadata={"category": "snow"},
    ),
    Document(
        page_content="""In the urban landscape, snow descended, transforming
            bustling streets into a winter wonderland, where the laughter of
            children echoed amidst the flurry of snowballs and the twinkle of
            holiday lights.""",
        metadata={"category": "snow"},
    ),
    Document(
        page_content="""Atop the rugged peaks, snow fell with an unyielding
            intensity, sculpting the landscape into a pristine alpine paradise,
            where the frozen crystals shimmered under the moonlight, casting a
            spell of enchantment over the wilderness below.""",
        metadata={"category": "snow"},
    ),
]


vector_store.add_documents(docs)

### 更新向量存储中的条目

要更新向量存储中的现有文档，请使用以下代码：

In [ ]:
updated_document = Document(
    page_content="qux", metadata={"source": "https://another-example.com"}
)

vector_store.update_documents(document_id="1", document=updated_document)

### 从向量存储中删除项目

要从向量存储中删除文档，请使用以下代码：

In [ ]:
vector_store.delete(ids=["3"])

## 查询向量存储

一旦您创建了向量存储并添加了相关文档，您很可能希望在链或代理运行时查询它。

### 直接查询

执行简单的相似性搜索可以这样做：

In [ ]:
results = vector_store.similarity_search(query="trees in the snow", k=1)
for doc in results:
    print(f"* {doc.page_content} [{doc.metadata}]")

如果您想执行相似性搜索并接收相应的分数，可以运行：

- TODO: 编辑然后运行代码单元以生成输出

In [ ]:
results = vector_store.similarity_search_with_score(query="trees in the snow", k=1)
for doc, score in results:
    print(f"* [SIM={score:3f}] {doc.page_content} [{doc.metadata}]")

### 元数据过滤
SingleStoreDB 通过允许用户基于元数据字段进行预过滤来增强和优化搜索结果，从而提升了搜索能力。此功能使开发人员和数据分析师能够微调查询，确保搜索结果精确地满足其要求。通过使用特定的元数据属性过滤搜索结果，用户可以缩小查询范围，仅关注相关的数据子集。

In [ ]:
query = "trees branches"
docs = vector_store.similarity_search(
    query, filter={"category": "snow"}
)  # Find documents that correspond to the query and has category "snow"
print(docs[0].page_content)

### 向量索引
通过利用 8.5 版本或更高版本的 SingleStore DB 中的 [ANN 向量索引](https://docs.singlestore.com/cloud/reference/sql-reference/vector-functions/vector-indexing/)，可以提高搜索效率。在创建向量存储对象时，通过设置 `use_vector_index=True` 可以激活此功能。此外，如果你的向量与默认的 OpenAI 嵌入大小 1536 存在维度差异，请确保相应地指定 `vector_size` 参数。

### 搜索策略
SingleStoreDB 提供多种搜索策略，每种策略都经过精心设计，以满足特定的用例和用户偏好。默认的 `VECTOR_ONLY` 策略利用向量运算（如 `dot_product` 或 `euclidean_distance`）直接计算向量之间的相似度得分，而 `TEXT_ONLY` 则采用基于 Lucene 的全文搜索，这对于以文本为中心的应用程序尤其有利。对于寻求平衡方法的**用户**，`FILTER_BY_TEXT` 首先根据文本相似度优化结果，然后再进行向量比较，而 `FILTER_BY_VECTOR` 优先考虑向量相似度，在评估文本相似度以获得最佳匹配之前进行结果过滤。值得注意的是，`FILTER_BY_TEXT` 和 `FILTER_BY_VECTOR` 都需要全文索引才能运行。此外，`WEIGHTED_SUM` 作为一种复杂的策略，通过对向量和文本相似度进行加权来计算最终相似度得分，但仅使用点积距离计算，并且也需要全文索引。这些多功能的策略使用户能够根据其独特的需求微调搜索，从而实现高效和精确的数据检索和分析。此外，SingleStoreDB 的混合方法，以 `FILTER_BY_TEXT`、`FILTER_BY_VECTOR` 和 `WEIGHTED_SUM` 策略为例，无缝地融合了基于向量和文本的搜索，以最大限度地提高效率和准确性，确保用户能够充分利用平台的强大功能来处理各种应用程序。

In [ ]:
from langchain_singlestore.vectorstores import DistanceStrategy

docsearch = SingleStoreVectorStore.from_documents(
    docs,
    embeddings,
    distance_strategy=DistanceStrategy.DOT_PRODUCT,  # Use dot product for similarity search
    use_vector_index=True,  # Use vector index for faster search
    use_full_text_search=True,  # Use full text index
)

vectorResults = docsearch.similarity_search(
    "rainstorm in parched desert, rain",
    k=1,
    search_strategy=SingleStoreVectorStore.SearchStrategy.VECTOR_ONLY,
    filter={"category": "rain"},
)
print(vectorResults[0].page_content)

textResults = docsearch.similarity_search(
    "rainstorm in parched desert, rain",
    k=1,
    search_strategy=SingleStoreVectorStore.SearchStrategy.TEXT_ONLY,
)
print(textResults[0].page_content)

filteredByTextResults = docsearch.similarity_search(
    "rainstorm in parched desert, rain",
    k=1,
    search_strategy=SingleStoreVectorStore.SearchStrategy.FILTER_BY_TEXT,
    filter_threshold=0.1,
)
print(filteredByTextResults[0].page_content)

filteredByVectorResults = docsearch.similarity_search(
    "rainstorm in parched desert, rain",
    k=1,
    search_strategy=SingleStoreVectorStore.SearchStrategy.FILTER_BY_VECTOR,
    filter_threshold=0.1,
)
print(filteredByVectorResults[0].page_content)

weightedSumResults = docsearch.similarity_search(
    "rainstorm in parched desert, rain",
    k=1,
    search_strategy=SingleStoreVectorStore.SearchStrategy.WEIGHTED_SUM,
    text_weight=0.2,
    vector_weight=0.8,
)
print(weightedSumResults[0].page_content)

### 查询（转换为检索器）

你也可以将向量存储转换为检索器，以便在你的链中使用。

In [ ]:
retriever = vector_store.as_retriever(search_kwargs={"k": 1})
retriever.invoke("trees in the snow")

## 多模态示例：利用 CLIP 和 OpenClip 嵌入

在多模态数据分析领域，整合图像和文本等不同类型的信息已变得日益重要。促进这种整合的一个强大工具是 [CLIP](https://openai.com/research/clip)，这是一种能够将图像和文本嵌入到共享语义空间的最先进模型。通过这样做，CLIP 能够通过相似性搜索跨不同模态检索相关内容。

为了说明这一点，让我们考虑一个旨在有效分析多模态数据的应用程序场景。在此示例中，我们利用 [OpenClip 多模态嵌入](/docs/integrations/text_embedding/open_clip) 的功能，该嵌入利用了 CLIP 的框架。借助 OpenClip，我们可以无缝地嵌入文本描述以及相应的图像，从而实现全面的分析和检索任务。无论是根据文本查询识别视觉上相似的图像，还是查找与特定视觉内容相关的文本片段，OpenClip 都能使用户能够以卓越的效率和准确性来探索和提取多模态数据中的见解。

In [ ]:
%pip install -U langchain openai lanchain-singlestore langchain-experimental

In [ ]:
import os

from langchain_experimental.open_clip import OpenCLIPEmbeddings
from langchain_singlestore.vectorstores import SingleStoreVectorStore

os.environ["SINGLESTOREDB_URL"] = "root:pass@localhost:3306/db"

TEST_IMAGES_DIR = "../../modules/images"

docsearch = SingleStoreVectorStore(OpenCLIPEmbeddings())

image_uris = sorted(
    [
        os.path.join(TEST_IMAGES_DIR, image_name)
        for image_name in os.listdir(TEST_IMAGES_DIR)
        if image_name.endswith(".jpg")
    ]
)

# Add images
docsearch.add_images(uris=image_uris)

## 使用检索增强生成

有关如何将此向量存储用于检索增强生成 (RAG) 的指南，请参阅以下部分：

- [教程](/docs/tutorials/)
- [操作指南：使用 RAG 进行问答](https://python.langchain.com/docs/how_to/#qa-with-rag)
- [检索概念文档](https://python.langchain.com/docs/concepts/retrieval)

## API 参考

有关 SingleStore 文档加载器的所有功能和配置的详细文档，请访问 GitHub 页面：[https://github.com/singlestore-labs/langchain-singlestore/](https://github.com/singlestore-labs/langchain-singlestore/)